# Wrapping Stata Code in Python

As a part of the Medicaid imputation project, I have a use case that involves in estimator that does not appear to be implemented in Python: the multivariate probit estimator.  (That being said, this could probably be tackled by a suitably designed implementation of [PyStan](https://github.com/stan-dev/pystan).) Essentially, we think that there is a relationship between the receipt of SNAP and Medicaid benefits (and other transfer benefits directed at low-income folks for that matter).  The most straightforward implementation seems to be contained by the `mvprobit` estimator in Stata.  This Notebook demonstrates how one might 1) use Python to deal with the data management, 2) pass the data to Stata for estimation, and 3) return the relevant results back into the Python namespace.

In [37]:
import numpy as np
import pandas as pd
from pandas import Series, DataFrame

import subprocess

%pylab inline

Populating the interactive namespace from numpy and matplotlib


`%matplotlib` prevents importing * from pylab and numpy


## Data Simulation

So that we may check our results against the model laid out in the article on [`mvprobit`](https://www.google.com/url?sa=t&rct=j&q=&esrc=s&source=web&cd=1&cad=rja&uact=8&ved=0ahUKEwiBl7OhwpjKAhUC7R4KHaLRAuQQFggdMAA&url=http%3A%2F%2Fwww.stata-journal.com%2Fsjpdf.html%3Farticlenum%3Dst0045&usg=AFQjCNHi1QVTZiK-YRagsQOuF37LmWht9w&sig2=ah1usVaro6ebRyEIGoiVog) housed in the Stata Journal, we will follow the same simulation procedure.  We are generating a set of four correlated equations with known parameter values so that we may identify the quality of the estimator.  The equations are taken directly from the article (Section 4.2).

In [38]:
#Set seed value (not that the same sequence will hold)
rand_seed=12309

#Set number of observations
nobs=5000

#Define correlation structure
vcov_R=np.array([[1.,25,.5,.75],[25,1.,.75,.5],[.5,.75,1.,.75],[.75,.5,.75,1.]])

#Set error means
u_means=np.zeros(4)

#Generate errors given the covariance structure
u_draws=np.random.multivariate_normal(u_means,vcov_R,nobs)

#Generate regressor data
x1 = np.random.uniform(size=nobs) - .5
x2 = np.random.uniform(size=nobs) + (1/3.)
x3 = 2*np.random.uniform(size=nobs) + .5
x4 = .5*np.random.uniform(size=nobs) - (1/3.)

#Generate latent response
y1_latent = .5 + 4*x1 + u_draws[:,0]
y2_latent = 3 + .5*x1 - 3*x2 + u_draws[:,1]
y3_latent = 1 - 2*x1 + .4*x2 - .75*x3 + u_draws[:,2]
y4_latent = -6 + 1*x1 -.3*x2 + 3*x3 - .4*x4 + u_draws[:,3]

#Generate observed binary response
y1 = np.where(y1_latent>0,1,0)
y2 = np.where(y2_latent>0,1,0)
y3 = np.where(y3_latent>0,1,0)
y4 = np.where(y4_latent>0,1,0)

#Define the number of draws
sim_draws=75

C:\Users\marvinw\AppData\Local\Continuum\Anaconda\lib\site-packages\ipykernel\__main__.py:14: RuntimeWarning: covariance is not positive-semidefinite.


With all of the relevant inputs defined, we will write them to disk, so that they may be used in the Stata `.do` file.

In [39]:
#Capture data in DF
model_data=DataFrame({'y1':y1,
                      'y2':y2,
                      'y3':y3,
                      'y4':y4,
                      'x1':x1,
                      'x2':x2,
                      'x3':x3,
                      'x4':x4})

#Capture parameters
model_params={'seed_in':rand_seed,
              'obs_in':nobs,
               'r_in':sim_draws}
for var in model_params.keys():
    model_data[var]=model_params[var]

#Write both DFs to disk
model_data.to_csv('stata_test_data.csv')
# model_params.to_csv('stata_test_params.csv')

In [40]:
model_data

,x1,x2,x3,x4,y1,y2,y3,y4,r_in,obs_in,seed_in
0,0.199767,1.175237,0.691626,-0.109307,1,1,1,0,75,5000,12309
1,-0.357995,0.584097,1.157628,-0.029932,0,0,1,0,75,5000,12309
2,0.069999,0.835527,0.791797,-0.002874,1,1,1,0,75,5000,12309
3,0.366108,0.926304,2.393065,0.152324,0,0,0,1,75,5000,12309
4,0.190336,0.769500,0.705648,-0.197538,1,1,1,0,75,5000,12309
5,-0.047170,0.348000,0.520795,-0.293764,1,1,1,0,75,5000,12309
6,0.300062,1.241612,1.102318,-0.165914,1,1,1,0,75,5000,12309
7,0.020545,1.242918,2.262506,-0.145513,1,0,0,1,75,5000,12309
8,0.168995,0.432498,2.228155,0.145769,1,1,0,1,75,5000,12309
9,-0.285402,0.984483,1.785408,-0.117445,1,1,1,0,75,5000,12309


Now we can use Python to call Stata from the command line.  Specifically, we need to get the Stata engine to run the `.do` file we have created.

In [41]:
!pwd
!cat py_stata_test.do

/home/MarvinW/work_scratch
*Change directory to work_scratch
cd C:\cygwin64\home\MarvinW\work_scratch

*Read in data
insheet using stata_test_data.csv

*Capture parameters from input data in mean estimation table
*(mean works because they are represented as repeated values across obs)
mean seed_in obs_in r_in

*Capture the results in an extractable matrix
matrix b=e(b)

*Assign each parameter to a local scalar for use in the routine
local seed_p=b[1,1]
local obs_p=b[1,2]
local r_p=b[1,3]

*Set random seed and number of obs
set seed `seed_p'
set obs `obs_p'

*Run multivariate probit model
mvprobit (y1=x1) (y2=x1 x2) (y3 = x1 x2 x3) (y4=x1 x2 x3 x4), dr(`r_p')

*Store output
estimates store model

*Write estimates to disk
esttab model using py_stata_test_results.csv

exit, STATA clear


In [42]:
# # Define Python function to launch a do-file 
# def dostata(dofile, *params):
#     '''Launch a do-file, given the fullpath to the do-file and a list of parameters.'''
#     import subprocess
#     cmd = ["stata-64", "do", dofile]
#     for param in params:
#         cmd.append(param)
#     return subprocess.call(cmd) 

# dostata('py_stata_test.do')

Stata is taking forever.  What if we tried a SAS routine instead?

In [43]:
!cat py_sas_test.sas

*Read in simulated data;
proc import datafile="C:\cygwin64\home\MarvinW\work_scratch\stata_test_data.csv"
	out=test
	dbms=csv
	replace;
run;

*Estimate multivariate probit model;
proc qlim data=test method=qn outest=mod_est covout;
	model y1=x1;
	model y2=x1 x2;
	model y3 = x1 x2 x3;
	model y4=x1 x2 x3 x4;
	endogenous y1 y2 y3 y4 ~ discrete;
	output out=mod_out marginal predicted prob;
run;

*Inspect results output;
data mod_est;
	set mod_est;
run;
data mod_out;
	set mod_out;
run;

*Write model estimates and output to disk;
proc export data=mod_est
	outfile="C:\cygwin64\home\MarvinW\work_scratch\mod_est.csv"
	dbms=CSV
	replace;
run;
proc export data=mod_out
	outfile="C:\cygwin64\home\MarvinW\work_scratch\mod_out.csv"
	dbms=CSV
	replace;
run;


In [44]:
# Define Python function to launch a do-file 
def dosas(dofile, *params):
    '''Launch a sas file, given the fullpath to the sas file and a list of parameters.'''
    import subprocess
    cmd = ["sas", dofile]
    for param in params:
        cmd.append(param)
    return subprocess.call(cmd) 

dosas('py_sas_test.sas')

0

That was much quicker.  Now we can explore the output.

In [45]:
#Read in model output
mod_out=pd.read_csv('mod_out.csv')

mod_out.head().T

,0,1,2,3,4
_,0.000000,1.000000,2.000000,3.000000,4.000000
x1,0.199767,-0.357995,0.069999,0.366108,0.190336
x2,1.175237,0.584097,0.835527,0.926304,0.769500
x3,0.691626,1.157628,0.791797,2.393065,0.705648
x4,-0.109307,-0.029932,-0.002874,0.152324,-0.197538
y1,1.000000,0.000000,1.000000,0.000000,1.000000
y2,1.000000,0.000000,1.000000,0.000000,1.000000
y3,1.000000,1.000000,1.000000,0.000000,1.000000
y4,0.000000,0.000000,0.000000,1.000000,0.000000
r_in,75.000000,75.000000,75.000000,75.000000,75.000000


So, let's unpack this a bit. `y1-y4` and `x1-x4` clearly capture the dependent and independent data, respectively.  `r_in, obs_in` and `seed_in` are all input parameters for the estimator.  Everything else was generated during estimation:

+ `P_y1-P_y4` capture the (discrete) predicted values for the dependent (based upon the coefficient estimates in the model spec);
+ `Meff*` capture the marginal effects for each regressor on the probability of a given observed value.  These differ by equation.  For example, `Meff_P1_y1.x1` features the marginal effect of `x1` on the probability of `y=1` in the first equation.  By contrast, `Meff_P2_y4.x3` features the marginal effect of `x3` on the probability of `y=2` in the fourth equation.
+ `Prob_y1-Prob_y4` capture the primary measure of interest, the continuous estimated probability of `y=1` for observation `i`.

What about the estimation output?

In [46]:
#Read in model output
mod_est=pd.read_csv('mod_est.csv')

mod_est

,_NAME_,_TYPE_,_STATUS_,y1_Intercept,y1_x1,y2_Intercept,y2_x1,y2_x2,y3_Intercept,y3_x1,...,y4_x1,y4_x2,y4_x3,y4_x4,_Rho_y1_y2,_Rho_y1_y3,_Rho_y1_y4,_Rho_y2_y3,_Rho_y2_y4,_Rho_y3_y4
0,NaN,PARM,0 Converged,1.421520e-01,1.213535e+00,0.943324,0.163842,-0.961477,1.018329e+00,-1.942970e+00,...,1.045231e+00,-2.919021e-01,2.997916e+00,-2.875369e-01,9.980581e-01,1.726421e-01,0.136096,1.775733e-01,1.475426e-01,7.235676e-01
1,NaN,STD,0 Converged,1.774307e-02,6.313894e-02,0.036875,0.062007,0.038931,7.851427e-02,7.050456e-02,...,9.379187e-02,8.992359e-02,8.539680e-02,1.747362e-01,8.313470e-04,2.306167e-02,0.031245,2.301458e-02,3.129200e-02,2.523936e-02
2,y1.Intercept,COV,0 Converged,3.148165e-04,2.770040e-05,0.000279,0.000006,-0.000015,3.366330e-05,-1.024528e-06,...,-1.116856e-06,1.738217e-06,1.919393e-06,-3.511913e-06,-4.744538e-07,-1.338940e-06,-0.000001,-1.718266e-06,-1.127077e-06,-1.189235e-07
3,y1.x1,COV,0 Converged,2.770040e-05,3.986526e-03,0.000423,0.003025,-0.000492,5.217891e-06,4.059625e-04,...,3.320844e-04,5.131259e-07,4.570243e-06,-6.313695e-06,1.861921e-07,-1.365600e-05,-0.000018,-1.896000e-05,-2.135900e-05,-5.681674e-06
4,y2.Intercept,COV,0 Converged,2.789725e-04,4.227659e-04,0.001360,-0.000289,-0.001260,1.060880e-04,-9.872430e-06,...,-1.969100e-05,-9.944000e-05,-1.703941e-06,8.829301e-06,1.306409e-06,-2.061700e-05,-0.000018,-2.388400e-05,-2.348300e-05,6.186983e-06
5,y2.x1,COV,0 Converged,6.153423e-06,3.024988e-03,-0.000289,0.003845,0.000329,-4.146400e-05,4.315089e-04,...,3.928419e-04,5.902560e-05,1.663190e-05,-8.051363e-06,-2.340284e-06,9.789318e-06,0.000004,8.196816e-06,5.763037e-06,-1.168400e-05
6,y2.x2,COV,0 Converged,-1.509200e-05,-4.917200e-04,-0.001260,0.000329,0.001516,-8.807500e-05,8.842343e-06,...,1.861770e-05,1.240094e-04,4.574218e-06,-2.016400e-05,-3.490985e-06,1.968710e-05,0.000016,2.901760e-05,3.115270e-05,-5.786847e-06
7,y3.Intercept,COV,0 Converged,3.366330e-05,5.217891e-06,0.000106,-0.000041,-0.000088,6.164490e-03,-6.561860e-04,...,-8.913300e-05,-1.430656e-03,-6.181840e-04,-4.076200e-05,1.705269e-07,2.306790e-05,0.000007,1.733520e-05,4.397627e-06,-7.267789e-06
8,y3.x1,COV,0 Converged,-1.024528e-06,4.059625e-04,-0.000010,0.000432,0.000009,-6.561860e-04,4.970894e-03,...,1.695020e-03,5.078990e-05,-2.748610e-04,4.026020e-05,-1.176236e-07,1.319400e-05,0.000035,1.073130e-05,3.231530e-05,5.644650e-05
9,y3.x2,COV,0 Converged,1.177744e-06,2.842488e-06,-0.000083,0.000054,0.000103,-3.517363e-03,-1.068570e-04,...,4.634610e-05,1.696453e-03,7.090600e-05,-3.256641e-06,-3.091783e-07,-2.343200e-05,-0.000012,-2.286200e-05,-1.286100e-05,-2.036800e-05


The model estimate DF contains parameter estimates (in our case, both $\beta$ and $\rho$) and their respective standard deviations. The output also provides a covariance matrix for the parameter estimates.  Precisely which value corresponds to what is driven by the descriptor contained in the `_TYPE_` variable.

In [47]:
0.0040297101**.5

0.06347999763705099